This notebook provides an automated end-to-end pipeline for creating a 3D reconstruction from images using [COLMAP](https://colmap.github.io/).

#### If you are using Ubuntu or have sudo access

In [ ]:
!sudo apt update
!sudo apt install colmap -y

#### For mac users

In [ ]:
brew install colmap

#### Main project folder

In [ ]:
!mkdir -p ~/sewer_reconstruction

#### Images folder where the images should be inputted

In [ ]:
! mkdir -p ~/sewer_reconstruction/images

#### Feature Matching - Exhaustive matcher

In [ ]:
!colmap exhaustive_matcher \
    --database_path ~/sewer_reconstruction/database.db \
    --SiftMatching.use_gpu 1

#### Folder for Sparse reconstruction

In [ ]:
!mkdir -p ~/sewer_reconstruction/sparse

#### Sparse Reconstruction or Mapping

In [ ]:
!colmap mapper \
    --database_path ~/sewer_reconstruction/database.db \
    --image_path ~/sewer_reconstruction/images \
    --output_path ~/sewer_reconstruction/sparse \
    --Mapper.num_threads 8

#### Dense reconstruction folder

In [ ]:
 !mkdir -p ~/sewer_reconstruction/dense

#### Undistort images

In [ ]:
!colmap image_undistorter \
    --image_path ~/sewer_reconstruction/images \
    --input_path ~/sewer_reconstruction/sparse/0 \
    --output_path ~/sewer_reconstruction/dense \
    --output_type COLMAP

#### Dense stereo matching

In [ ]:
!colmap patch_match_stereo \
    --workspace_path ~/sewer_reconstruction/dense \
    --workspace_format COLMAP \
    --PatchMatchStereo.geom_consistency true

#### Stereo fusion - point cloud

In [ ]:
!colmap stereo_fusion \
    --workspace_path ~/sewer_reconstruction/dense \
    --workspace_format COLMAP \
    --input_type geometric \
    --output_path ~/sewer_reconstruction/dense/fused.ply

#### Mesh reconstruction or Poisson Meshing

In [ ]:
!colmap poisson_mesher \
    --input_path ~/sewer_reconstruction/dense/fused.ply \
    --output_path ~/sewer_reconstruction/dense/poisson_mesh.ply